In [ ]:
!pip install langchain
!pip install openai
!pip install pypdf
!pip install glob
!pip install tiktoken

ERROR: Could not find a version that satisfies the requirement glob (from versions: none)
ERROR: No matching distribution found for glob
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.7 MB/s eta 0:00:00


In [ ]:
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import PyPDFLoader
from langchain import OpenAI, PromptTemplate
import glob
import os
from langchain.indexes import VectorstoreIndexCreator
from langchain.document_loaders import PyPDFDirectoryLoader

In [ ]:
from secret_key import openapi_key

In [ ]:
os.environ['OPENAI_API_KEY'] = ""

In [ ]:
pdfs_folder = "/content/f"

In [ ]:
for pdf_file in glob.glob(pdfs_folder + "/*.pdf"):

    print(pdf_file)

/content/f/2304.11062.pdf
/content/f/2109.01134.pdf
/content/f/2304.11477.pdf


In [ ]:
pdf_file = "/content/f/2304.11062.pdf"
pdf_file2 = "/content/f/2304.11477.pdf"
pdf_file3 = "/content/f/2109.01134.pdf"


In [ ]:
llm = OpenAI(temperature=0.2)
loader = PyPDFLoader(pdf_file3)
docs = loader.load_and_split()

In [ ]:
loader = PyPDFLoader(pdf_file)
docs = loader.load_and_split()
chain = load_summarize_chain(llm, chain_type="map_reduce")
summary = chain.run(docs)

In [ ]:
summary

' This study explores the use of recurrent memory augmentation in pre-trained transformer models to extend input context length while maintaining high retrieval accuracy. The proposed approach allows for storing information in memory for sequences up to two million tokens, with potential applications in natural language understanding and large-scale context processing. The study introduces a new model, the Recurrent Memory Transformer, which shows strong performance on longer sequences and can be combined with other methods for improved efficiency. The study also discusses the use of memory-augmented neural networks in various tasks and the potential for broader applications.'

In [ ]:
type(docs)

list

In [ ]:
len(docs)

21

In [ ]:
docs[19]

Document(page_content='Table 1: RMT can be successfully combined with parameter-\nefficient methods (parallel adapter, LoRA). Results for language\nmodeling on the Arxiv dataset for Pythia-70m model.\nMODEL LOSS\nADAPTER ONLY 41.43\nADAPTER + RMT-1 SEG 10.31\nADAPTER + L ORA + RMT-1 SEG 7.30\nADAPTER + L ORA + RMT-2 SEG 6.97\nRMT offers the flexibility of incorporation various cost-\nefficient training methods, which greatly enhances its prac-\ntical applicability, especially when computational resources\nare limited.', metadata={'source': '/content/f/2304.11062.pdf', 'page': 12})

In [ ]:
for i in docs:

    print(len(i.page_content))


4000
975
3953
1586
3971
775
3194
3373
3987
376
3972
728
3994
1828
3962
1961
819
3542
2578
489


In [ ]:
def summarize_pdfs_from_folder(pdfs_folder):
    summaries = []
    for pdf_file in glob.glob(pdfs_folder + "/*.pdf"):
        loader = PyPDFLoader(pdf_file)
        docs = loader.load_and_split()
        chain = load_summarize_chain(llm, chain_type="map_reduce")
        summary = chain.run(docs)
        print("Summary for: ", pdf_file)
        print(summary)
        print("\n")
        summaries.append(summary)

    return summaries

In [ ]:
def custom_summary(pdf_folder, custom_prompt):
    summaries = []
    for pdf_file in glob.glob(pdf_folder + "/*.pdf"):
        loader = PyPDFLoader(pdf_file)
        docs = loader.load_and_split()
        prompt_template = custom_prompt + """

        {text}

        SUMMARY:"""
        PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])
        chain = load_summarize_chain(llm, chain_type="map_reduce",
                                    map_prompt=PROMPT, combine_prompt=PROMPT)
        summary_output = chain({"input_documents": docs},return_only_outputs=True)["output_text"]
        summaries.append(summary_output)

    return summaries

In [ ]:
summaries = summarize_pdfs_from_folder("/content/f")

Summary for:  /content/f/2304.11062.pdf
 This study explores the use of recurrent memory augmentation in pre-trained transformer models to extend input context length while maintaining high retrieval accuracy. The proposed method allows for storing information in memory for sequences of up to two million tokens, with potential to enhance long-term dependency handling in natural language tasks. The study also discusses previous research on memory in neural architectures and introduces a new model, the Recurrent Memory Transformer (RMT), which has shown promising results in tasks such as long-range language modeling and theorem proving. The article also discusses various techniques and advancements in parameter-efficient transfer learning and the use of memory-augmented neural networks for long-range sequence modeling. The appendix provides details on the training process and compares the efficiency of RMT with full attention models. Overall, RMT has been shown to be a faster and more me

In [ ]:
summaries

[' This study explores the use of recurrent memory augmentation in pre-trained transformer models to extend input context length while maintaining high retrieval accuracy. The proposed method allows for storing information in memory for sequences of up to two million tokens, with potential to enhance long-term dependency handling in natural language tasks. The study also discusses previous research on memory in neural architectures and introduces a new model, the Recurrent Memory Transformer (RMT), which has shown promising results in tasks such as long-range language modeling and theorem proving. The article also discusses various techniques and advancements in parameter-efficient transfer learning and the use of memory-augmented neural networks for long-range sequence modeling. The appendix provides details on the training process and compares the efficiency of RMT with full attention models. Overall, RMT has been shown to be a faster and more memory-efficient option for handling lon

In [ ]:
# Python!
loader = PyPDFDirectoryLoader("/content/f")

docs = loader.load()

# Create the vector store index
index = VectorstoreIndexCreator().from_loaders([loader])

In [ ]:
len(docs)

34

In [ ]:
loader

In [ ]:
docs[0]

Document(page_content='Scaling Transformer to 1M tokens and beyond with RMT\nAydar Bulatov1, Yuri Kuratov2,1, Yermek Kapushev2, Mikhail Burtsev3\n1Neural Networks and Deep Learning Lab, MIPT, Dolgoprudny, Russia\n2AIRI, Moscow, Russia\n3London Institute for Mathematical Sciences, London, UK\n{bulatov.as,yurii.kuratov }@phystech.edu, mb@lims.ac.uk\nAbstract\nA major limitation for the broader scope of problems solv-\nable by transformers is the quadratic scaling of computational\ncomplexity with input size. In this study, we investigate the\nrecurrent memory augmentation of pre-trained transformer\nmodels to extend input context length while linearly scaling\ncompute. Our approach demonstrates the capability to store\ninformation in memory for sequences of up to an unprece-\ndented two million tokens while maintaining high retrieval\naccuracy. Experiments with language modeling tasks show\nperplexity improvement as the number of processed input\nsegments increases. These results undersc

In [ ]:
docs[-1]

Document(page_content='Autoregressive task and motion planning with llms as translators and\ncheckers,” arXiv preprint arXiv:2306.06531 , 2023.\n[46] K. Valmeekam, S. Sreedharan, M. Marquez, A. Olmo, and S. Kamb-\nhampati, “On the planning abilities of large language models (a\ncritical investigation with a proposed benchmark),” arXiv preprint\narXiv:2302.06706 , 2023.\n[47] T. Silver, V . Hariprasad, R. S. Shuttleworth, N. Kumar, T. Lozano-\nP´erez, and L. P. Kaelbling, “PDDL planning with pretrained\nlarge language models,” in NeurIPS 2022 Foundation Models\nfor Decision Making Workshop , 2022. [Online]. Available: https:\n//openreview.net/forum?id=1QMMUB4zfl\n[48] V . Pallagani, B. Muppasani, K. Murugesan, F. Rossi, L. Horesh,\nB. Srivastava, F. Fabiano, and A. Loreggia, “Plansformer: Generating\nsymbolic plans using transformers,” arXiv preprint arXiv:2212.08681 ,\n2022.\n[49] D. Arora and S. Kambhampati, “Learning and leveraging verifiers to\nimprove planning capabilities of pre-t

In [ ]:
docs[20]

Document(page_content='Wang H, Ge S, Lipton Z, Xing EP (2019) Learning robust\nglobal representations by penalizing local predictive power.\nIn: NeurIPS\nXiao J, Hays J, Ehinger KA, Oliva A, Torralba A (2010) Sun\ndatabase: Large-scale scene recognition from abbey to zoo.\nIn: CVPR\nYuan L, Chen D, Chen YL, Codella N, Dai X, Gao J,\nHu H, Huang X, Li B, Li C, et al. (2021) Florence: A\nnew foundation model for computer vision. arXiv preprint\narXiv:211111432\nZhang Y, Jiang H, Miura Y, Manning CD, Langlotz CP\n(2020) Contrastive learning of medical visual represen-\ntations from paired images and text. arXiv preprint\narXiv:201000747\nZhong Z, Friedman D, Chen D (2021) Factual probing is\n[mask]: Learning vs. learning to recall. In: NAACL\nZhou B, Lapedriza A, Khosla A, Oliva A, Torralba A (2017)\nPlaces: A 10 million image database for scene recognition.\nIEEE transactions on pattern analysis and machine intel-\nligence 40(6):1452–1464\nZhou K, Liu Z, Qiao Y, Xiang T, Loy CC (2021) Do

In [ ]:
len(docs)

34

In [ ]:
index

VectorStoreIndexWrapper(vectorstore=<langchain_community.vectorstores.inmemory.InMemoryVectorStore object at 0x7cbeafcced10>)

In [ ]:
query = "who is Ms dhoni?"

index.query(query)

" I don't know."